In [13]:
import polars as pl
from polars import col as c
import networkx as nx

from config import settings
import json
import os
from datetime import datetime, UTC
import datetime as dt

from utility.polars_operation import generate_uuid_col
from utility.parser_utility import (
    add_table_to_changes_schema,
    generate_connectivity_table,
    generate_random_uuid,
)
from utility.general_function import pl_to_dict

from twindigrid_changes.schema import ChangesSchema
from twindigrid_sql.schema.enum import (
    MeasurementClass,
    MeasurementPhase,
    MeasurementColumn,
    SubstationType,
    TerminalSide,
)
from twindigrid_sql.entries.source import (
    SCADA,
    CONVENTIONAL_METER,
    GRID_LOAD,
    SCADA,
    ESTIMATED,
)


from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    INDIRECT_FEEDER,
    BUSBAR_SECTION,
    ENERGY_CONSUMER,
)
from twindigrid_sql.entries.measurement_type import ENERGY, ACTIVE_POWER, REACTIVE_POWER
from twindigrid_sql.entries.unit_symbol import WATTHOUR, WATT

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))
# os.getcwd()

# Import data from matlab

In [14]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))

In [15]:
parameter_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_parameter"])
nodedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_node_data"])
powerdata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Power_data"])
linedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Line_data"])
result_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_result"])
# nodedata_distflow = nodedata_distflow.with_columns(c("Snom").cast(pl.Int8))
# # To have base value (need lenght of line), not from matlab !
# line_data_from_input_file: pl.DataFrame = pl.read_excel(
#     file_names["Line_Data_From_Input_File"]
# )

# Add node number to power data
powerdata_distflow = powerdata_distflow.with_row_index(
    "node_number", offset=1
)  # offset=1 because slack bus is 0 and no power on it
powerdata_distflow = powerdata_distflow.with_columns(c("node_number").cast(pl.Int64))
# Create a topology dataframe with basic topology information

df_topology = nodedata_distflow.select(
    c("index").alias("node_number"),
    c("Vnom"),
)

# Add the power data to the topology dataframe with node as key
df_topology = df_topology.join(
    powerdata_distflow, on="node_number", how="full", coalesce=True
)
nodedata_distflow

index,indexLines_1,indexLines_2,indexLines_3,Vnom,Snom,powerFactor_1,powerFactor_2,Bnode,SM,Annual
i64,i64,i64,i64,i64,str,f64,i64,i64,f64,f64
0,1,2,3,400,null,0.8,1,0,NaN,0.0
1,4,5,null,400,null,0.8,1,0,NaN,0.0
2,6,7,8,400,null,0.8,1,0,NaN,0.0
3,9,null,null,400,null,0.8,1,0,NaN,0.0
4,null,null,null,400,null,0.8,1,0,12.0,0.0
…,…,…,…,…,…,…,…,…,…,…
53,null,null,null,400,null,0.8,1,0,NaN,3095.956989
54,55,56,null,400,null,0.8,1,0,NaN,0.0
55,57,null,null,400,null,0.8,1,0,NaN,0.0


# Set missing value for equipment

In [16]:
### Set missing value for equipment
# Fake value for the length of the branch
base_length = 1
# Fake value for the switch state
switch_state = False
switch_type = "locked_switch"
switch_command = "unknown"

# Initialisation
default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)
heartbeat = datetime.now(UTC)
changes_schema = ChangesSchema()

## Connectivity node table

In [17]:
# Generate the node dict with uuid for each node
connectivity_node: dict[float, str] = pl_to_dict(
    df_topology.select(
        c("node_number"),
        c("node_number").pipe(generate_uuid_col, added_string="node_").alias("uuid"),
    )
)
## Add the cn_fk to the topology dataframe
df_topology = df_topology.with_columns(
    c("node_number").replace_strict(connectivity_node, default=None).alias("cn_fk")
)
df_topology

node_number,Vnom,Pload,Qload,cn_fk
i64,i64,f64,f64,str
0,400,null,null,"""df941fce-ceda-5874-ab63-5c8af9…"
1,400,0.0,0.0,"""ba84d70a-80d7-590e-b112-f9c4b5…"
2,400,0.0,0.0,"""078656ed-79f8-53a1-a67a-bb8f53…"
3,400,0.0,0.0,"""c2247320-9fc2-538a-ba64-3ac70e…"
4,400,0.0,0.0,"""af72457f-f983-5eeb-a635-0609f4…"
…,…,…,…,…
53,400,0.0,0.0,"""4505ed8e-f087-5ee2-8c67-775daa…"
54,400,0.000035,-0.000012,"""23bc00b6-0e27-5e6d-a02e-dda5e9…"
55,400,0.0,0.0,"""b1d51456-8036-5737-accc-1103d2…"


In [18]:
low_voltage_limit = 380
high_voltage_limit = 420

node = df_topology.with_columns(
    c("Vnom").alias("base_voltage_fk"),
    pl.lit(low_voltage_limit).alias("low_voltage_limit"),
    pl.lit(high_voltage_limit).alias("high_voltage_limit"),
    c("cn_fk").alias("uuid"),
)
base_voltage = node.with_columns(
    c("base_voltage_fk").alias("nominal_voltage"),
    pl.lit("LV").alias("type"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "ConnectivityNode": node,
    "BaseVoltage": base_voltage,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema,
    new_tables_pl=new_tables_pl,
    raw_table_name="ConnectivityNode",
)
changes_schema

2025-01-13 13:10:50 B01BPC14 utility.parser_utility[110679] WARNING 58 double values over 58 detected in ConnectivityNode_base_voltage for columns nominal_voltage


ChangesSchema(heartbeat=shape: (0, 2)
┌──────┬───────────────────┐
│ diff ┆ heartbeat         │
│ ---  ┆ ---               │
│ str  ┆ datetime[μs, UTC] │
╞══════╪═══════════════════╡
└──────┴───────────────────┘, resource=shape: (0, 12)
┌──────┬──────┬───────────────────┬───────────────────┬───┬──────┬───────────┬──────────┬───────┐
│ diff ┆ uuid ┆ start_heartbeat   ┆ end_heartbeat     ┆ … ┆ name ┆ feeder_fk ┆ metadata ┆ owner │
│ ---  ┆ ---  ┆ ---               ┆ ---               ┆   ┆ ---  ┆ ---       ┆ ---      ┆ ---   │
│ str  ┆ str  ┆ datetime[μs, UTC] ┆ datetime[μs, UTC] ┆   ┆ str  ┆ str       ┆ str      ┆ str   │
╞══════╪══════╪═══════════════════╪═══════════════════╪═══╪══════╪═══════════╪══════════╪═══════╡
└──────┴──────┴───────────────────┴───────────────────┴───┴──────┴───────────┴──────────┴───────┘, equipment=shape: (0, 2)
┌──────┬──────┐
│ diff ┆ uuid │
│ ---  ┆ ---  │
│ str  ┆ str  │
╞══════╪══════╡
└──────┴──────┘, terminal=shape: (0, 4)
┌──────┬───────┬──────┬───────

## Branch

In [19]:
# branch :pl.DataFrame =


# Current and other line parameter in pu

# Filter to take only branch, connection_type == 2
branch = linedata_distflow.filter(c("connection_type") == 2).select(
    ("line_" + c("line_number").cast(pl.String)).alias("dso_code"),
    c("i_pu").alias("current_limit"),
    c("r_pu"),
    c("x_pu"),
    c("b_pu"),
    # Need column name non null value for validation of the schema
    pl.lit(base_length).alias("length"),  # km
    pl.lit(BRANCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    c("line_number").pipe(generate_uuid_col, added_string=BRANCH).alias("uuid"),
    # Generate uuid for each terminal of branch with node uuid
    c("node_from").replace_strict(connectivity_node, default=None).alias("t1"),
    c("node_to").replace_strict(connectivity_node, default=None).alias("t2"),
    # Need column name for validation of the schema
    pl.lit(None).alias("t1_container_fk"),
    pl.lit(None).alias("t2_container_fk"),
)
branch_parameter_event: pl.DataFrame = branch.with_columns(
    c("uuid").alias("eq_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    c("r_pu").alias("r"),
    c("x_pu").alias("x"),
    c("b_pu").alias("b"),
    c("start").alias("timestamp"),
    pl.lit(heartbeat).alias("heartbeat"),
    pl.lit(SCADA).alias("source_fk"),  # ??? why or why not ?
).with_columns(pl.lit(0.0).alias(col) for col in ["g", "r0", "x0", "b0", "g0"])

new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": branch,
    "Equipment": branch,
    "Branch": branch,
    "BranchParameterEvent": branch_parameter_event,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="branch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="branch"
)

## Energy consumer

In [20]:
default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)


# Power in PU
energy_consumer = df_topology.with_columns(
    ("node_number_" + c("node_number").cast(pl.String)).alias("dso_code"),
    pl.lit(ENERGY_CONSUMER).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit("unknown").alias("profile_type"),
    pl.lit(0).alias("rated_p"),  # Symbol: P_ec_nom, Unit: kW
    c("cn_fk")
    .pipe(generate_random_uuid)
    .alias("uuid"),  # Generate random uuid on a random column
    c("Pload").replace(0, None).alias("node_with_consumer"),
).drop_nulls(
    "node_with_consumer"
)  # Remove node without consumption
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": energy_consumer,
    "Equipment": energy_consumer,
    "EnergyConsumer": energy_consumer,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="energy_consumer"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="energy_consumer"
)

## Measurement

Active power

In [21]:
## Add the uuid of the node to the power data
measurement = energy_consumer.select(
    c("uuid")
    .pipe(generate_random_uuid)
    .alias(
        "uuid"
    ),  # Generate random uuid on a column without importance (don't work with pl.lit)
    c("uuid").alias("resource_fk"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(MeasurementClass.SPAN.value).alias("concrete_type"),
    pl.lit(MeasurementPhase.ABC.value).alias("phase"),
    pl.lit(MeasurementColumn.DOUBLE.value).alias("column_type"),
    pl.lit(CONVENTIONAL_METER).alias("source_fk"),
    # pl.lit(60*60*24*365).alias("default_period"),
    pl.lit(ACTIVE_POWER).alias("measurement_type"),
    pl.lit("pu").alias("unit_symbol"),
    pl.lit(1).alias("unit_multiplier"),
    c("Pload").alias("double_value"),
)
measurement_span = measurement.with_columns(
    c("uuid").alias("measurement_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    pl.lit(datetime(2022, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("start"),
    pl.lit(datetime(2023, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("end"),
)

new_tables_pl: dict[str, pl.DataFrame] = {
    "Measurement": measurement,
    "MeasurementSpan": measurement_span,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="meter_id"
)

Reactive power

In [22]:
## Add the uuid of the node to the power data
measurement = energy_consumer.select(
    c("uuid")
    .pipe(generate_random_uuid)
    .alias(
        "uuid"
    ),  # Generate random uuid on a column without importance (don't work with pl.lit)
    c("uuid").alias("resource_fk"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(MeasurementClass.SPAN.value).alias("concrete_type"),
    pl.lit(MeasurementPhase.ABC.value).alias("phase"),
    pl.lit(MeasurementColumn.DOUBLE.value).alias("column_type"),
    pl.lit(CONVENTIONAL_METER).alias("source_fk"),
    # pl.lit(60*60*24*365).alias("default_period"),
    pl.lit(REACTIVE_POWER).alias("measurement_type"),
    pl.lit("pu").alias("unit_symbol"),
    pl.lit(1).alias("unit_multiplier"),
    c("Qload").alias("double_value"),
)
measurement_span = measurement.with_columns(
    c("uuid").alias("measurement_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    pl.lit(datetime(2022, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("start"),
    pl.lit(datetime(2023, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("end"),
)

new_tables_pl: dict[str, pl.DataFrame] = {
    "Measurement": measurement,
    "MeasurementSpan": measurement_span,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="meter_id"
)

## Switch

In [23]:
# Filter to take only switch, connection_type == 3
switch = linedata_distflow.filter(c("connection_type") == 3).select(
    ("line_" + c("line_number").cast(pl.String)).alias("dso_code"),
    pl.lit(SWITCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(switch_state).alias("normal_open"),
    pl.lit(switch_type).alias("type"),
    pl.lit(switch_command).alias("command"),
    # Generate uuid for each terminal of branch with node uuid
    c("node_from").replace_strict(connectivity_node, default=None).alias("t1"),
    c("node_to").replace_strict(connectivity_node, default=None).alias("t2"),
    # Need column name for validation of the schema
    pl.lit(None).alias("t1_container_fk"),
    pl.lit(None).alias("t2_container_fk"),
    c("line_number").pipe(generate_uuid_col, added_string=SWITCH).alias("uuid"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": switch,
    "Equipment": switch,
    "Switch": switch,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="switch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=switch, raw_data_table="switch"
)

In [24]:
# Begin time of the data from matlab (from main_FC.ipynb before)
str(datetime(2020, 4, 4, 23, 00, 0, 0, UTC) - dt.timedelta(hours=192))

'2020-03-27 23:00:00+00:00'

## Distflow

In [25]:
# changes_schema.connectivity
# changes_schema.measurement["resource_fk"][0]
# changes_schema.branch.filter(c("uuid") == "df941fce-ceda-5874-ab63-5c8af9bec38b")
# changes_schema.connectivity.filter(
#     c("cn_fk").is_in(changes_schema.measurement["resource_fk"])
# )
# changes_schema.energy_consumer.join(changes_schema.resource, on="uuid", how="inner")
list(changes_schema.__dict__.keys())

['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

In [26]:
changes_schema.branch.join(changes_schema.resource, on="uuid", how="inner")

diff,uuid,section,current_limit,length,is_underground,diff_right,start_heartbeat,end_heartbeat,start,end,dso_code,concrete_class,name,feeder_fk,metadata,owner
str,str,f64,f64,f64,bool,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str
"""+""","""b4fdf239-cfe2-5453-91c3-84c1b2…",null,0.024942,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_1""","""branch""",null,null,null,null
"""+""","""3edce6fc-e164-5b3e-9b0d-8a1c86…",null,0.026327,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_2""","""branch""",null,null,null,null
"""+""","""c173d7f3-f7ca-5f44-98d8-63c1aa…",null,0.003048,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_4""","""branch""",null,null,null,null
"""+""","""c33afba9-0379-5d6a-9ca1-743cf7…",null,0.024942,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_5""","""branch""",null,null,null,null
"""+""","""020bd95c-9320-5560-9627-4bbe19…",null,0.026327,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_6""","""branch""",null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""d9975d35-54c3-5993-b044-835e1b…",null,0.003048,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_53""","""branch""",null,null,null,null
"""+""","""e46fa9df-d633-5149-96b4-c9cccc…",null,0.024942,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_54""","""branch""",null,null,null,null
"""+""","""34edd04e-a827-5cf8-b233-43d285…",null,0.024942,1.0,null,"""+""",2025-01-13 12:10:50.226883 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_55""","""branch""",null,null,null,null


In [27]:
changes_schema.base_voltage

diff,nominal_voltage,type,low_voltage_limit,high_voltage_limit
str,i32,str,i32,i32
"""+""",400,"""LV""",380,420


In [28]:
changes_schema.branch.join(
    changes_schema.connectivity, left_on="uuid", right_on="eq_fk", how="inner"
) ??????????????????????????????????????????????????????????????????????????????????????????

SyntaxError: invalid syntax (47334181.py, line 3)

## Import data to changes schema

In [ ]:
# energy_consumer.filter(c("node_number") == 5)
print(branch.filter(c("dso_code") == "line_5"))
print(
    changes_schema.terminal.select(c("eq_fk")).filter(
        c("eq_fk") == "c33afba9-0379-5d6a-9ca1-743cf7c3512d"
    )
)

In [149]:
changes_schema.terminal.select(c("eq_fk")).filter(
    c("eq_fk") == "c33afba9-0379-5d6a-9ca1-743cf7c3512d"
)
changes_schema.connectivity.with_columns(
    c("cn_fk").is_unique().alias("unique")
)  # Give true for the slack node...
test2 = changes_schema.connectivity.filter(
    c("cn_fk").is_first_distinct() == True
)  # Give true for the slack node...

In [150]:
v_slack_sq = pow(parameter_distflow["Vslack"][0], 2)
test = changes_schema.connectivity  # .filter(c("side") == "t1")
# line_data :
# "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
# "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
# "P": [0, 1, 2, 1, 4, 3, 6, 5],
# "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
# "p_line": [0] * 8,

In [ ]:
dist_flow_df : pl.DataFrame = 
iteration = 0
# parameter_distflow
# ((norm(Vnode_sq(:,2) - Vnode_sq(:,1)) > tol  && iteration < maxIteration))
tol = parameter_distflow["tol"][0]
max_iteration = parameter_distflow["maxIteration"][0]


x = 
while x < tol and iteration < max_iteration:
    print(iteration)
    iteration += 1
# Qload_augmented = Qload - Bnode .* Vnode_sq(:,1);

In [ ]:
def sum_downstream_power(col: pl.Expr, df: pl.DataFrame):
    return col.map_elements(
        lambda x: df.filter(c("upstream") == x)["p_line"].sum(), return_dtype=pl.Float64
    )


def calculate_line_power(df: pl.DataFrame):
    return (c("downstream").pipe(sum_downstream_power, df=df) + c("P")) * (1 + c("F"))


def sum_power(df: pl.DataFrame, lv: int):

    return df.with_columns(
        pl.when(c("lv") == lv)
        .then(calculate_line_power(df=df))
        .otherwise(c("p_line"))
        .alias("p_line")
    )


# UP Use for each powerflow
# Down Use only one time
def get_node_level(G: nx.DiGraph) -> dict:
    level_mapping: dict = {}
    for node in reversed(list(nx.topological_sort(G))):
        if not len(list(G.successors(node))):
            level_mapping[node] = 0
        else:
            level_mapping[node] = max(level_mapping[n] for n in G.successors(node)) + 1
    return level_mapping


line_data: pl.DataFrame = pl.DataFrame(
    {
        "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
        "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
        "P": [0, 1, 2, 1, 4, 3, 6, 5],
        "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        "p_line": [0] * 8,
    }
)

grid = nx.DiGraph()

_ = line_data.drop_nulls(subset="upstream").with_columns(
    pl.struct(c("upstream"), c("downstream")).map_elements(
        lambda x: grid.add_edge(x["upstream"], x["downstream"]), return_dtype=pl.Struct
    )
)
level_mapping: dict = get_node_level(G=grid)
line_data = line_data.with_columns(
    c("downstream").replace_strict(level_mapping, default=None).alias("lv")
)

for i in range(line_data["lv"].max() + 1):
    line_data = sum_power(df=line_data, lv=i)

print(line_data.sort("lv"))